In [1]:
import datasets
import torch

from transformers import BertTokenizer, BertModel

In [2]:
DataFilePath = "../personality_dataset"
tokenizer = BertTokenizer.from_pretrained('../bert-base-uncase')

In [3]:
labels = ["ENTJ", "ENTP", "ENFJ", "ENFP", "ESFJ", "ESFP", "ESTJ", "ESTP", "INTP", "INTJ", "INFP", "INFJ", "ISFP", "ISFJ", "ISTP", "ISTJ"]

In [4]:
import torch.utils
import numpy as np


class Dataset(torch.utils.data.Dataset):
    def __init__(self, data, number):
        self.trian = datasets.load_from_disk(DataFilePath+"/"+data)
        self.labels = [labels.index(label) for label in self.trian["personality"][0 : number if number < len(self.trian["personality"]) else -1]]
        self.texts = [tokenizer(person.replace("|||", "[SEP]"),
                                padding="max_length",
                                max_length=512,
                                truncation=True,
                                return_tensors="pt")
                      for person in self.trian["content"][0 : number if number < len(self.trian["content"]) else -1]]
    
    def classes(self):
        return self.labels
    
    def __len__(self):
        return len(self.labels)
    
    def get_batch_labels(self, idx):
        return np.array(self.labels[idx])
    
    def get_batch_texts(self, idx):
        return self.texts[idx]
    
    def __getitem__(self, idx):
        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)
        return batch_texts, batch_y

In [5]:
from torch import nn

class BertClassifier(nn.Module):
    def __init__(self, dropout=0.5):
        super(BertClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('../bert-base-uncase')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 16)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):
        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)
        return final_layer

In [6]:
from torch.optim import Adam
from tqdm import tqdm

def train(model, train_data, val_data, learning_rate, epochs, number):
    
    train, val = Dataset(train_data, number), Dataset(val_data, -1)
    
    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)
    
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    
    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=learning_rate)

    if use_cuda:
            model = model.cuda()
            criterion = criterion.cuda()
    
    for epoch_num in range(epochs):
        
        total_acc_train = 0
        total_loss_train = 0
        
        for train_input, train_label in tqdm(train_dataloader):

            train_label = train_label.to(device)
            mask = train_input['attention_mask'].to(device)
            input_id = train_input['input_ids'].squeeze(1).to(device)
            
            output = model(input_id, mask)
            
            batch_loss = criterion(output, train_label.to(torch.int64))
            total_loss_train += batch_loss.item()
            
            acc = (output.argmax(dim=1) == train_label).sum().item()
            total_acc_train += acc
            
            model.zero_grad()
            batch_loss.backward()
            optimizer.step()
            
            
            total_acc_val = 0
            total_loss_val = 0
            
            with torch.no_grad():
                
                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)
                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label.to(torch.int64))
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'''Epochs: {epoch_num + 1} 
                | Train Loss: {total_loss_train / len(train_data): .3f} 
                | Train Accuracy: {total_acc_train / len(train_data): .3f} 
                | Val Loss: {total_loss_val / len(val_data): .3f} 
                | Val Accuracy: {total_acc_val / len(val_data): .3f}''')   

In [7]:
EPOCHS = 1
model = BertClassifier()
LR = 1e-6
number = 100

train(model, "train", "valid",  LR, EPOCHS, number)

c:\Users\王蔚昕\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
  2%|▏         | 1/50 [03:17<2:41:12, 197.40s/it]

Epochs: 1 
                | Train Loss:  0.620 
                | Train Accuracy:  0.000 
                | Val Loss:  871.500 
                | Val Accuracy:  33.400


  4%|▍         | 2/50 [06:26<2:34:08, 192.69s/it]

Epochs: 1 
                | Train Loss:  1.207 
                | Train Accuracy:  0.000 
                | Val Loss:  868.304 
                | Val Accuracy:  41.000


  6%|▌         | 3/50 [09:35<2:29:34, 190.94s/it]

...

Epochs: 1 
                | Train Loss:  26.565 
                | Train Accuracy:  1.400 
                | Val Loss:  847.783 
                | Val Accuracy:  66.200


 98%|█████████▊| 49/50 [2:39:16<03:18, 198.05s/it]

Epochs: 1 
                | Train Loss:  27.139 
                | Train Accuracy:  1.400 
                | Val Loss:  848.910 
                | Val Accuracy:  69.200


100%|██████████| 50/50 [2:42:35<00:00, 195.12s/it]

Epochs: 1 
                | Train Loss:  27.724 
                | Train Accuracy:  1.400 
                | Val Loss:  849.141 
                | Val Accuracy:  64.600
